In [1]:
import sqlite3
import pandas as pd
andmebaasi_nimi = "v33_koondkorpus_sentences_verb_pattern_obl_20241002-130310.db"
tabel = "kohakaanded" # tabel, kus on ainult kohakäänded alles jäetud (feats_filtreerimine.ipynb tulemus)

In [2]:
examples_given = 5 # mitu rida iga lemma kohta välja valitakse
count_min = 5 # minimaalne lemma esinemiste kord, et seda välja valitaks
conn = sqlite3.connect(andmebaasi_nimi)
query = f"""
WITH RankedRows AS (
    -- Järjestame samad lemmad kasutades random'it ning määrame example_num väärtuse
    SELECT 
        *, 
        ROW_NUMBER() OVER (PARTITION BY lemma ORDER BY RANDOM()) AS example_num,
        COUNT(*) OVER (PARTITION BY lemma) AS total_count -- Mitu korda antud lemma esineb
    FROM {tabel}
),
ValidLemmas AS (
    -- Jätame alles ainult lemmad, mis esinevad vähemalt count_min korda
    SELECT lemma
    FROM RankedRows
    GROUP BY lemma
    HAVING COUNT(*) >= {count_min}
),
RandomLemmas AS (
    -- Valime juhuslikult lemmad
    SELECT lemma
    FROM ValidLemmas
    ORDER BY RANDOM()
    -- LIMIT 1000 (siin saab määrata mitu erinevat lemmat valitakse)
)
SELECT RankedRows.*
FROM RankedRows
JOIN RandomLemmas USING (lemma)
WHERE example_num <= {examples_given}
ORDER BY total_count, lemma, example_num;
"""


df = pd.read_sql_query(query, conn)
conn.close()
df # dataframe, kus on iga valitud lemma kohta examples_given näidet 

,id,head_id,loc,loc_rel,deprel,form,lemma,feats,parent_loc,pos,case,number,proper,example_num,total_count
0,16914226,9042088,5,2,obl,01-l,01,"ad,card,l,sg",None,N,ad,sg,0,1,5
1,33842830,18347728,1,-1,obl,01-lt,01,"abl,card,l,sg",None,N,abl,sg,0,2,5
2,23059612,12407197,5,2,obl,01-l,01,"ad,card,l,sg",None,N,ad,sg,0,3,5
3,48580405,26546238,11,1,obl,01-st,01,"card,el,l,sg",None,N,el,sg,0,4,5
4,51743905,28551088,8,3,obl,01-st,01,"card,el,l,sg",None,N,el,sg,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229995,29381721,15908272,18,1,obl,meil,mina,"ad,pl",None,P,ad,pl,0,1,209164
229996,10880759,5820169,3,1,obl,meil,mina,"ad,pl",None,P,ad,pl,0,2,209164
229997,21488130,11525505,21,1,obl,mullegi,mina,"all,sg",None,P,all,sg,0,3,209164
229998,31378415,16997757,1,-1,obl,Mulle,mina,"all,sg",None,P,all,sg,0,4,209164


In [3]:
conn = sqlite3.connect(andmebaasi_nimi)

db_df = pd.read_sql("SELECT id, sentence_id FROM transaction_head", conn) # saame sentance_id

conn.close()

# lisame sentence_id oma dataframe'ile
df = df.merge(db_df, left_on="head_id", right_on="id", how="left")

# eemaldame üleliigse id veeru
df.drop(columns=["id_y"], inplace=True)

df.head(50)

,id_x,head_id,loc,loc_rel,deprel,form,lemma,feats,parent_loc,pos,case,number,proper,example_num,total_count,sentence_id
0,16914226,9042088,5,2,obl,01-l,01,"ad,card,l,sg",None,N,ad,sg,0,1,5,5638210
1,33842830,18347728,1,-1,obl,01-lt,01,"abl,card,l,sg",None,N,abl,sg,0,2,5,11452252
2,23059612,12407197,5,2,obl,01-l,01,"ad,card,l,sg",None,N,ad,sg,0,3,5,7738885
3,48580405,26546238,11,1,obl,01-st,01,"card,el,l,sg",None,N,el,sg,0,4,5,17296885
4,51743905,28551088,8,3,obl,01-st,01,"card,el,l,sg",None,N,el,sg,0,5,5,18907961
5,48405559,26440568,12,2,obl,1.90st,1.90,"card,el,l,sg",None,N,el,sg,0,1,5,17221900
6,27327188,14758582,9,4,obl,1.90-le,1.90,"all,card,l,sg",None,N,all,sg,0,2,5,9182197
7,28843248,15608077,19,3,obl,1.90st,1.90,"card,el,l,sg",None,N,el,sg,0,3,5,9727368
8,41289266,22316899,3,1,obl,1.90st,1.90,"card,el,l,sg",None,N,el,sg,0,4,5,14018110
9,35416751,19181957,3,1,obl,1.90-l,1.90,"ad,card,l,sg",None,N,ad,sg,0,5,5,11985432


In [4]:
# kasutame lausete andmebaasi
conn = sqlite3.connect("v33_koondkorpus_sentences_sentences_20250220-130121.db")

db_df = pd.read_sql("SELECT id, text FROM sentences", conn) # saame laused (text)
conn.close()
# lisame sentence_id põhjal dataframe'i ridadele vastava lause
df = df.merge(db_df, left_on="sentence_id", right_on="id", how="left")
# eemaldame üleliigse id veeru
df.drop(columns=["id"], inplace=True)
# muudame veerude järjekorda
uus_järjekord = ["lemma", "sentence_id", "head_id", "example_num","total_count","feats","case", "number", "proper","deprel","form", "loc", "loc_rel", "parent_loc", "pos","id_x", "text"]
df = df[uus_järjekord]
df

,lemma,sentence_id,head_id,example_num,total_count,feats,case,number,proper,deprel,form,loc,loc_rel,parent_loc,pos,id_x,text
0,01,5638210,9042088,1,5,"ad,card,l,sg",ad,sg,0,obl,01-l,5,2,None,N,16914226,Tuntumatest nimedest esinevad ARS 01-l Anish K...
1,01,11452252,18347728,2,5,"abl,card,l,sg",abl,sg,0,obl,01-lt,1,-1,None,N,33842830,"01-lt pudenes latt napilt alla , ometi oli Kas..."
2,01,7738885,12407197,3,5,"ad,card,l,sg",ad,sg,0,obl,01-l,5,2,None,N,23059612,Tuntumatest nimedest esinevad ARS 01-l Anish ...
3,01,17296885,26546238,4,5,"card,el,l,sg",el,sg,0,obl,01-st,11,1,None,N,48580405,nagu jokker korra juba ütles .. « ma ei ütleks...
4,01,18907961,28551088,5,5,"card,el,l,sg",el,sg,0,obl,01-st,8,3,None,N,51743905,Alates 33. leheküljest algavad uuesti koodide ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229995,mina,9916343,15908272,1,209164,"ad,pl",ad,pl,0,obl,meil,18,1,None,P,29381721,"Aga kui lugeda kokku kõik laevaomaniku kulud ,..."
229996,mina,3614263,5820169,2,209164,"ad,pl",ad,pl,0,obl,meil,3,1,None,P,10880759,Praegu oleks meil tuleviku kohta vaja rohkem i...
229997,mina,7165281,11525505,3,209164,"all,sg",all,sg,0,obl,mullegi,21,1,None,P,21488130,Toomas Liivi väide Nietzschest kui “ puuduvast...
229998,mina,10604189,16997757,4,209164,"all,sg",all,sg,0,obl,Mulle,1,-1,None,P,31378415,"Mulle meenus , kuidas miski tõmbas lennukit se..."


In [5]:
# salvestame saadud dataframe'i andmebaasi
conn = sqlite3.connect("BT_andmed.db") 

df.to_sql("table", conn, if_exists="replace", index=False)
conn.close()

# Lisa - vali saadud andmetest esimesed 5000 rida

In [6]:
# valime välja esimesed 5000
'''
conn = sqlite3.connect("BT_andmed.db")

db_df = pd.read_sql('SELECT * FROM "table" LIMIT 5000;', conn)
conn.close()
db_df
'''

'\nconn = sqlite3.connect("BT_andmed.db")\n\ndb_df = pd.read_sql(\'SELECT * FROM "table" LIMIT 5000;\', conn)\nconn.close()\ndb_df\n'

In [7]:
# salvestame saadud read andmebaasi faili
'''
conn = sqlite3.connect("BT_andmed_5000.db") 

df.to_sql("table", conn, if_exists="replace", index=False)
conn.close()
'''

'\nconn = sqlite3.connect("BT_andmed_5000.db") \n\ndf.to_sql("table", conn, if_exists="replace", index=False)\nconn.close()\n'

In [8]:
#TODO leida enda 500 sõna laused
# juhuslik 100sõna * 5 näidet